In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with WMF-S model and simulated feedback

In [2]:
from pathlib import Path
import numpy as np

import teaser.util as util
import teaser.evaluation as evaluation

from teaser.algorithm.wmf_s import WMF_S

## Datasets

In [3]:
## Uncomment corresponding directory.
# DATA_DIR = Path('../../data/ML20M/')
# DATA_DIR = Path('../../data/Amazon Video Games/')
DATA_DIR = Path('../../data/Publiq/')

In [4]:
# Shouldn't need to change this info
INTERACTIONS_TRAIN_PATH = DATA_DIR / 'interactions.train.csv'
INTERACTIONS_VALIN_PATH = DATA_DIR / 'interactions.valin.csv'
INTERACTIONS_VALOUT_PATH = DATA_DIR / 'interactions.valout.csv'
INTERACTIONS_TESTIN_PATH = DATA_DIR / 'interactions.testin.csv'
INTERACTIONS_TESTOUT_PATH = DATA_DIR / 'interactions.testout.csv'

METADATA_PATH = DATA_DIR / 'features.csv'

ITEM_ID = 'itemId'
USER_ID = 'userId'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = WMF_S

# ML20M
# L2 = 500
# ALPHA = 10

# AVG
# L2 = 1
# ALPHA = 150

# Publiq
L2 = 2000
ALPHA = 10

HYPERPARAMS = {'l2': L2, 'alpha': ALPHA}
HYPERPARAMS

{'l2': 2000, 'alpha': 10}

## Parse data

In [6]:
S, tags = util.parse_metadata(METADATA_PATH, item_id=ITEM_ID)
n, t = S.shape  # amount of items and tags

X = util.parse_interactions(INTERACTIONS_TRAIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

Xtest_in = util.parse_interactions(INTERACTIONS_TESTIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)
Xtest_out = util.parse_interactions(INTERACTIONS_TESTOUT_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

In [7]:
# # debug
# d_items = 1000
# S = S[:d_items,:]
# tags = tags[:d_items]
# X = X[:,:d_items]
# Xtest_in = Xtest_in[:,:d_items]
# Xtest_out = Xtest_out[:,:d_items]

## Evaluate model with optimal hyperparams on test set

In [8]:
%%time
alg = ALG(**HYPERPARAMS)
alg.fit(X, S)

CPU times: user 9.76 s, sys: 2.94 s, total: 12.7 s
Wall time: 656 ms


In [9]:
print("Performance on test set with simulation")
for POS_FEEDBACK in [1, 2]:
    for STRENGTH in [3]:
        print("Pos feedback count:", POS_FEEDBACK, "with strength (out of 5):", STRENGTH)
        score = evaluation.simulate(alg, Xtest_in, Xtest_out, S, pos_feedback=POS_FEEDBACK, strength=STRENGTH, repeats=3)
    print()

Performance on test set with simulation
Pos feedback count: 1 with strength (out of 5): 3
Evaluating with 1500 users
Average Recall@20 0.394
Average Recall@100 0.629
Average nDCG@100 0.261

Pos feedback count: 2 with strength (out of 5): 3
Evaluating with 1500 users
Average Recall@20 0.5
Average Recall@100 0.732
Average nDCG@100 0.336

